In [1]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
price_response_df = pd.read_csv('/Users/youziya/Library/CloudStorage/OneDrive-YorkUniversity/OMIS 6000 Models and Applications in Operational Reseach/Assignment 2/price_response.csv')

a)

In [3]:
from scipy.optimize import minimize
import numpy as np

# Coefficients for the Basic and Advanced versions
a1, b1 = 35234.545786, 45.896450
a2, b2 = 37790.240832, 8.227794

# Define the objective function to be minimized (negative revenue since we're using a minimization function)
def objective(p):
    p1, p2 = p
    return -((p1 * (a1 - b1 * p1)) + (p2 * (a2 - b2 * p2)))

# Initial guess (starting prices)
p_initial = [100, 150]  # Arbitrary initial prices for Basic and Advanced versions

# Constraints
# p2 >= p1 (Advanced price should be higher than Basic)
# Prices p1, p2 >= 0 are inherently considered by the bounds parameter
constraints = [{'type': 'ineq', 'fun': lambda p: p[1] - p[0]}]  # Advanced version price - Basic version price >= 0

# Bounds for p1 and p2 (assuming a realistic minimum price of 0 and a maximum price that could be reasonably set for laptops)
bounds = [(0, None), (0, None)]

# Solve the optimization problem
result = minimize(objective, p_initial, method='SLSQP', bounds=bounds, constraints=constraints)

# Extract the optimal prices
p_optimal = result.x

p_optimal, -result.fun  # Display optimal prices and maximum revenue


(array([ 383.8440553 , 2296.50832013]), 50154984.20934893)

b)

In [4]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Define the gradient of the objective function
def gradient(p):
    p1, p2 = p
    grad_p1 = a1 - 2*b1*p1  # Gradient with respect to p1
    grad_p2 = a2 - 2*b2*p2  # Gradient with respect to p2
    return np.array([grad_p1, grad_p2])

# Initialize variables
p = np.array([0.0, 0.0])  # Starting prices
step_size = 0.001
tolerance = 1e-6
max_iterations = 10000  # Safety mechanism to prevent infinite loops

# Begin the projected gradient descent loop
for _ in range(max_iterations):
    # Compute the current gradient
    grad = gradient(p)
    
    # Gradient step
    p_temp = p + step_size * grad
    
    # Set up and solve the quadratic programming problem for projection
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model(env=env) as m:
            # Define variables with lower bounds of 0
            p_vars = m.addVars(2, lb=0, name="p")
            # Objective: minimize the distance to the gradient step
            m.setObjective(sum((p_vars[i] - p_temp[i]) * (p_vars[i] - p_temp[i]) for i in range(2)), GRB.MINIMIZE)
            # Constraint: p2 >= p1
            m.addConstr(p_vars[1] >= p_vars[0], "c")
            # Solve the model
            m.optimize()
            
            # Update prices based on the projection result
            p_new = np.array([p_vars[i].X for i in range(2)])
    
    # Check for convergence: if the change in prices is less than the tolerance, stop
    if np.linalg.norm(p_new - p) < tolerance:
        print("Convergence achieved.")
        break
    
    # Update prices for the next iteration
    p = p_new

# Print the final optimal prices
print(f"Optimal Prices: p1 = {p[0]:.4f}, p2 = {p[1]:.4f}")


Convergence achieved.
Optimal Prices: p1 = 383.8483, p2 = 2296.4989


c)

In [7]:

# Actual intercepts and sensitivities for each product version in each line
ai = [
    [35234.54579, 37790.24083, 35675.33322],  # Line 1: Basic, Advanced, Premium
    [37041.38038, 36846.14039, 35827.02375],  # Line 2: Basic, Advanced, Premium
    [39414.26632, 35991.95146, 39313.31703],  # Line 3: Basic, Advanced, Premium
]
bi = [
    [45.89644971, 8.227794173, 7.58443641],  # Line 1 Sensitivities
    [9.033166404, 4.427869206, 2.629060015],  # Line 2 Sensitivities
    [2.421483918, 4.000512401, 2.296622373],  # Line 3 Sensitivities
]

# Create a new model
model = gp.Model("MaximizeRevenue")

# Create decision variables for the prices of each product version
prices = model.addVars(3, 3, lb=0, name="Price")

# Set the objective function to maximize total revenue
revenue = gp.quicksum(prices[i, j] * (ai[i][j] - bi[i][j] * prices[i, j])
                      for i in range(3) for j in range(3))
model.setObjective(revenue, GRB.MAXIMIZE)

# Add constraints for price ordering within each product line
for i in range(3):
    model.addConstr(prices[i,0] <= prices[i,1], f"PriceOrder_{i}_1")
    model.addConstr(prices[i,1] <= prices[i,2], f"PriceOrder_{i}_2")

# Solve the model
model.optimize()

# Print the optimal prices and calculate the suggested optimal revenue
if model.status == GRB.OPTIMAL:
    for i in range(3):
        for j in range(3):
            print(f"Optimal Price for Line {i+1} Product {j+1}: {prices[i, j].X:.2f}")
    print(f"Optimal Total Revenue: {model.ObjVal:.2f}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 9 columns and 12 nonzeros
Model fingerprint: 0xe01f23c0
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.34101234e+08  1.18618490e+09  5.00e+02 2.21e+02  7.06e+0

d)

In [18]:
import gurobipy as gp
from gurobipy import GRB

# Actual intercepts and sensitivities for each product version in each line
ai = [
    [35234.54579, 37790.24083, 35675.33322],  # Line 1: Basic, Advanced, Premium
    [37041.38038, 36846.14039, 35827.02375],  # Line 2: Basic, Advanced, Premium
    [39414.26632, 35991.95146, 39313.31703],  # Line 3: Basic, Advanced, Premium
]
bi = [
    [45.89644971, 8.227794173, 7.58443641],  # Line 1 Sensitivities
    [9.033166404, 4.427869206, 2.629060015],  # Line 2 Sensitivities
    [2.421483918, 4.000512401, 2.296622373],  # Line 3 Sensitivities
]

# Create a new model
model = gp.Model("MaximizeRevenue")

# Create decision variables for the prices of each product version
prices = model.addVars(3, 3, lb=0, name="Price")

# Set the objective function to maximize total revenue
revenue = gp.quicksum(prices[i, j] * (ai[i][j] - bi[i][j] * prices[i, j])
                      for i in range(3) for j in range(3))
model.setObjective(revenue, GRB.MAXIMIZE)

# Add constraints for price ordering within each product line (i)
# [Previous setup and optimization code]

# Add constraints for increasing prices within each product line
for i in range(3):
    model.addConstr(prices[i, 0] <= prices[i, 1], "Basic<=Advanced within line %d" % i)
    model.addConstr(prices[i, 1] <= prices[i, 2], "Advanced<=Premium within line %d" % i)

# Add new constraints for increasing prices across comparable versions
for j in range(3):
    model.addConstr(prices[0, j] <= prices[1, j], "EvoTech<=InfiniteEdge for version %d" % j)
    model.addConstr(prices[1, j] <= prices[2, j], "InfiniteEdge<=FusionBook for version %d" % j)

# Solve the model with additional constraints
model.optimize()

# Output the optimal prices and the optimal revenue
if model.status == GRB.OPTIMAL:
    print(f"Optimal Total Revenue with New Constraints: ${model.ObjVal:,.2f}")
    for i in range(3):
        for j in range(3):
            print(f"Optimal Price for Line {i+1} Version {['Basic', 'Advanced', 'Premium'][j]}: ${prices[i, j].X:.2f}")
else:
    print("Model did not solve to optimality.")


# Solve the model
model.optimize()

# Print the optimal prices and calculate the suggested optimal revenue
if model.status == GRB.OPTIMAL:
    for i in range(3):
        for j in range(3):
            print(f"Optimal Price for Line {i+1} Product {j+1}: {prices[i, j].X:.2f}")
    print(f"Optimal Total Revenue: {model.ObjVal:.2f}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 9 columns and 24 nonzeros
Model fingerprint: 0xf41c9d8f
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.34101234e+08  1.18618490e+09  5.00e+02 1.83e+02  7.59e

e)the most sensible prices to implement would ideally follow the logic of (d), as there's a clear differentiation in pricing both within each product line and across product lines for the same versions, reflecting both product value and strategic market positioning. This approach ensures logical price progression within each product line also positions different product lines in the market, reflecting their value and brand prestige accurately. It's a more comprehensive and market-aware strategy that likely aligns better with consumer expectations and competitive dynamics.

f) the consumer behaviour might change nd hard for us to predict. there are also external factors and economic condition.
actual pricing strategies should consider market realities, competitor pricing, and cost. also this is a non-linear model, it may not capture the linear demand like real demand behaviour.